In [ ]:
class Spider:
    '''
    This is the constructor class to which you can pass a bunch of parameters. 
    These parameters are stored to the class instance variables so that the
    class functions can access them later.
    
    no_of_pages: number of pages to scrape
    '''
    
    def __init__(self, no_of_pages=1):
        